# Database practical

With prof. Brian Marsden you were introduced to databases. Now let's get our hands dirty.

* Aim 1: Derp around with a database
* Aim: learn about how ChEMBL is structured and how to query it in order to understand how data can be stored and retrieved from a database.


To get started run the next cell to install the necessary packages.
NB. If you are using a Jupyter notebook you need to install SQLite3 a different way.

Questions, while you wait:

1. I am technoluddite chemists and I like Excel spreadsheets: Convince me why I should use a database instead of Excel.

In [ ]:
## Part 1: Derping around with a database

In [ ]:
## Install Sqlite3
# This is a Black magic level mega hack, don't worry about it.
# It installs mamba without resetting the kernel as condacolab does
# this is will install python 3.12
!wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
!bash Mambaforge-Linux-x86_64.sh -bfp /usr/local && clear
!mamba config --set auto_update_conda false
!mamba install -y conda-forge::sqlite && clear
import sys  # this as of Oct 2024 is 3.10
v = f'{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}'
!mamba create -y -n matched "python={v}" && clear
sys.path.append(f'/usr/local/envs/matched/lib')

First. Let's create a database and insert some data into it.
However, after reading XKCD 327, we want to try our hand at SQL injection.

![Exploits](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [ ]:
%%bash
# %% is called a 'cell magic'
rm -rf test.db
sqlite3 test.db "CREATE TABLE students (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL);"
sqlite3 test.db ".tables"
export STUDENT_NAME='Alice';
sqlite3 test.db "INSERT INTO students (name) VALUES ('${STUDENT_NAME}');"
sqlite3 test.db "SELECT * FROM students;"
export STUDENT_NAME="Robert'; DROP TABLE students; --";
sqlite3 test.db "INSERT INTO students (name) VALUES ('${STUDENT_NAME}');"
sqlite3 test.db "SELECT * FROM students;"

Bobby drop tables strikes again! Let's try in Python.

In [ ]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()
# create the table, what is the SQL command again?
cursor.execute('''
👾👾👾
''')

# let's check the tables, but more pythonically
from typing import List

def list_tables(conn: sqlite3.Connection) -> List[str]:
    cursor: sqlite3.Cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

def get_table_schema(conn: sqlite3.Connection, table_name: str):
    cursor: sqlite3.Cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    return schema

list_tables(conn)

Question: What is the difference between Connection and Cursor?

In [ ]:
# let's insert some data, the _safe_ way
user_name = "Alice"
cursor.execute('INSERT INTO students (name) VALUES (?)', (user_name,))
user_name = "BOBY'); DROP TABLE students; --"
cursor.execute('INSERT INTO students (name) VALUES (?)', (user_name,))
conn.commit()
# let's check the data
print(cursor.execute('SELECT * FROM students').fetchall())
# Nooooo Bobby!

# let's insert some data, the _unsafe_ way
unsafe_query = f"INSERT INTO students (name) VALUES ('{user_name}')"
cursor.execute(unsafe_query)

#conn.close()
list_tables(conn)

What happened in ? Why did the table not drop?

The SQLite3 module uses SQL queries, and is not ORM. What was an ORM again? Can you name a common module with an ORM behaviour?
How about a Python web framework that uses database that has an ORM?

In [ ]:
# ## Sidequest 1
# can you make the student table have their age and grade?

👾👾👾

# ## Sidequest 2
# can you make the student table have a foreign key to a table called 'courses'?
# PostGres can handle arrays, not SQLite3, so for ease, these students are only in one course.

👾👾👾

# ## Sidequest 4
# Get a list of all the students in the course 'Chemistry' (WHERE clause)

👾👾👾

# ## Sidequest 3
# The students filled in their details themselves.
# What are the problems with this approach? How can you mitigate them?

👾👾👾

## Part 2: ChEMBL

### Basics of ChEMBL

* Database: ChEMBL
* Type: curated activity data of medically relevant compounds
* URL: https://www.ebi.ac.uk/chembl/

Unusually, in ChEMBL API schema is different from the SQL schema. The API schema is more user-friendly, while the SQL schema is how the relational database is structured.

 To scare you, here is the schema of the ChEMBL 20 database (current version is 34):
 ![link](https://uc16f6cf72beac49c60508fb94bf.previews.dropboxusercontent.com/p/thumb/ACcijf_-UBCcd5-SDxc91SF_0QXyHbKW7g-xoXSCWf44GjS9xCvlU02fpxBwIZpE_9g56VY_YwwHHC_aBJkMPFiVejMV2mAcXeyVqup0haEszUYo5qIRX_Wgr58pquxwGQgo9oaTgGhHy8StIcxvXdz631A_vEgoXr7qOVjstMOwwAYXEn_j9eAiH3FSTNqWEfhcK8mf-cktqUIR5KpUE_ETgxfCv0Bn-00f1NFKW_4HM1ifrF7KCLYcvzBQLeBbu0Y5Vx8BiXS6-QcDiTtgXTJqKslaokWFlu2NPKXOrU08N-SNzXQ7fPBPC9_9-Pr1Zlvwg6C5Lj44p67vyH3eADaJ_iVTKl-in-Rn0gbF_wNOQEVvub4seUQu0ToaixGLPz5odMW0XzkRx3gNzLj48c9m/p.png?is_prewarmed=true). In reality you will only need a few tables.
 
In the [TeachOpenCADD tutorial](https://github.com/volkamerlab/teachopencadd/blob/master/teachopencadd/talktorials/T001_query_chembl/talktorial.ipynb) ChEMBL is interacted via an API. In this practical we will interact with the database directly, because teaching.

This API vs. DB difference is unusual as many frameworks like Django or FastAPI, etc. have views that are easily implemented using the database schema. A common auto-generated documentation system for web app views is [Swagger, now called OpenAPI](https://swagger.io/specification/). What was the last API you looked at the documentation for and was it a familiar format?

### Hypothetical scenario
This is for discussion purposes —and if you will need to use the DB in future. We will not do this in this practical,
as the DB is 43 GB.

Download the database.

**Where**. In the HPC class, we saw how a rack cabinet is organised
generally with local storage physically within each rack unit,
while in the baremetal/VM/countainer running therein
shared storage is mounted. The former storage is much faster and the latter is visible to all nodes. Say we assigned the variables $SHARED_STORAGE and $LOCAL_STORAGE to these two and want to keep a backup of compressed database in the former and the decompressed in the latter.
This is because it is faster to read from the local storage than the shared storage!
(The /tmp folder is typically 'ephemeral' storage, which gets wiped after a reboot).

**Checksum** Some of you may remember the joy of getting a stack of floppy disks of Mandrake 6 or Slackware 3 from a friend of a friend after a lot of effort possibly paying them in a currency that no longer exists (for me, Italian Lira), but then one diskette had an integrity error. Twenty years later everything is mostly better, but with large downloads this still and does happen. This is why we have checksums. 


```bash
# 4.6 GB
curl https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_34_sqlite.tar.gz --output $SHARED_STORAGE/chembl_34_sqlite.tar.gz
# Checksum
echo 'Computed checksum'
sha256sum $SHARED_STORAGE/chembl_34_sqlite.tar.gz;
curl https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/checksums.txt --output /tmp/checksums.txt
echo 'Expected checksum'
grep chembl_34_sqlite.tar.gz /tmp/checksums.txt | sha256sum -c -
# 43 GB
tar -xzvf $SHARED_STORAGE/chembl_34_sqlite.tar.gz -C $LOCAL_STORAGE/
mv $LOCAL_STORAGE/chembl_34/chembl_34_sqlite/chembl_34.db $LOCAL_STORAGE/chembl_34.db
rm -rf $LOCAL_STORAGE/chembl_34  # remove the documentation
```

Now. We have the database.
We can look at https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/schema_documentation.html and connect to it.


```python
import sqlite3

conn = sqlite3.connect('/tmp/chembl_34/chembl_34_sqlite/chembl_34.db')
cursor = conn.cursor()
tables = list_tables(conn)
print(tables)
```

    ['action_type', 'assay_type', 'chembl_id_lookup', 'confidence_score_lookup', 'curation_lookup', 'chembl_release', 'source', 'relationship_type', 'target_type', 'variant_sequences', 'bioassay_ontology', 'data_validity_lookup', 'activity_smid', 'activity_stds_lookup', 'assay_classification', 'atc_classification', 'bio_component_sequences', 'component_sequences', 'protein_classification', 'domains', 'go_classification', 'structural_alert_sets', 'products', 'frac_classification', 'hrac_classification', 'irac_classification', 'research_stem', 'organism_class', 'patent_use_codes', 'usan_stems', 'version', 'cell_dictionary', 'docs', 'target_dictionary', 'tissue_dictionary', 'molecule_dictionary', 'activity_supp', 'component_class', 'component_domains', 'component_go', 'component_synonyms', 'structural_alerts', 'defined_daily_dose', 'product_patents', 'protein_class_synonyms', 'research_companies', 'assays', 'compound_records', 'binding_sites', 'biotherapeutics', 'compound_properties', 'compound_structural_alerts', 'compound_structures', 'molecule_atc_classification', 'molecule_frac_classification', 'molecule_hierarchy', 'molecule_hrac_classification', 'molecule_irac_classification', 'molecule_synonyms', 'target_components', 'target_relations', 'activities', 'assay_class_map', 'assay_parameters', 'biotherapeutic_components', 'drug_indication', 'drug_mechanism', 'drug_warning', 'formulations', 'metabolism', 'site_components', 'activity_properties', 'activity_supp_map', 'indication_refs', 'ligand_eff', 'mechanism_refs', 'metabolism_refs', 'predicted_binding_domains', 'warning_refs', 'sqlite_stat1']

Now, I want to compare small molecules that have neural effects vs the rest. But this snippet get stuck at the COMPOUND_STRUCTURES table. Why?:

```python
import json
from pathlib import Path
import pandas as pd
import operator
import sqlite3

conn = sqlite3.connect('/tmp/chembl_34/chembl_34_sqlite/chembl_34.db')
df = pd.read_sql_query("SELECT * FROM MOLECULE_DICTIONARY", conn)
print('Loaded')

# Get ATC classification
atc = pd.read_sql_query("SELECT * FROM MOLECULE_ATC_CLASSIFICATION", conn)
atc['level1'] = atc.level5.apply(operator.itemgetter(0))
regno2atc = pd.pivot_table(atc, index='molregno', values='level1', aggfunc=''.join).level1.apply(set).apply(''.join).to_dict()
del atc
df['atc_level1s'] = df.molregno.map(regno2atc).fillna('')
print('Loaded ATC')

# Get SMILES
coms = pd.read_sql_query("SELECT * FROM COMPOUND_STRUCTURES", conn)
regno2smiles = coms.set_index('molregno').canonical_smiles.to_dict()
df['smiles'] = df.molregno.map(regno2smiles).fillna('')
del coms
print('Added SMILES')

conn.close()
```

Any guess why the memory requirement is too high?
How can I fix this?




🤩 ワクワク! (Exciting!) Your SQL query worked!
You are so good at this that I will export the data and let you finish the analysis.

```python
df['is_neuro'] = df.atc_level1s.str.contains('N')
smol_df = df.loc[(df.molecule_type == 'Small molecule') & ((df.atc_level1s != '') | (~df.max_phase.isna())) & (df.smiles != '')].copy()
smol_df.to_pickle('ChEMBL-subset.pkl.gz')
```


In [ ]:
!pip install -q pandas plotly rdkit
import plotly.express as px
from IPython.display import display

df = pd.read_pickle('https://github.com/matteoferla/DTC-compchem-practical/raw/refs/heads/main/2024-notebooks/data/ChEMBL-subset.pkl.gz')

# How many molecules are there for the two values in `is_neuro` column?
👾👾👾


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# Add a column `mol` with the RDKit molecule object
df['mol'] = df.smiles.apply(👾👾👾)  # what function should go here? fun(smiles) -> mol

# add some useful descriptors

df['HAC'] = df.mol.apply(👾👾👾)
# remember that a method is normally called `instance.method()`, but actually `class.method(instance, )` will work too
df['logP'] = ... # cf. https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html

# More automated? See https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html

In [ ]:
# Max Phase is a column that indicates the phase of the clinical trial
df.max_phase.sort_values()\
             .map({4: 'Approved',
                  3: 'Phase 3 Clinical Trials',
                  2: 'Phase 2 Clinical Trials',
                  1: 'Phase 1 Clinical Trials',
                  0.5: 'Early Phase 1 Clinical Trials',
                  -1: 'Clinical Phase',
                  float('nan'): 'No data'})\
              .value_counts()
# any cool ideas what to do with this?

### They say drugs are getting bigger
the column `usan_year` is when the USAN name was assigned.
USAN name is a non-proprietary name for a drug.
INN name is the international non-proprietary name, they match most of the time.
acetaminophen vs. paracetamol, epinephrine vs. adrenaline, and a couple more.
Let's see if the molecules are getting bigger over time.
They say drugs are getting bigger.
the column `usan_year` is when the USAN name was assigned.
USAN name is the American non-proprietary name for a drug.
INN name is the international non-proprietary name, they match most of the time.
The exceptions are acetaminophen vs. paracetamol, epinephrine vs. adrenaline, and a couple more.


In [ ]:
# Let's see if the molecules are getting bigger over time,
# by plotting what? vs what?

👾👾👾

### Blood brain barrier

A big challenge in drug design is to get the drug to the target,
especially if it's the brain

In [ ]:
# Are neuro molecules greasier?

👾👾👾

In [ ]:
# what do they look like?
print('Neuro')
subbed = smol_df.loc[~smol_df.is_neuro].sample(20)
display( Draw.MolsToGridImage(subbed.mol, legends=subbed.pref_name.astype(str).to_list(), molsPerRow=6) )

print('Not neuro')
subbed = smol_df.loc[~smol_df.is_neuro].sample(20)
display( Draw.MolsToGridImage(subbed.mol, legends=subbed.pref_name.astype(str).to_list(), molsPerRow=6) )

# how about the top ('head') and bottom ('tail') of the logP?

👾👾👾

## It's natural!

Are any of the neurologically active molecules natural products?

In [ ]:
# let's draw a grid image of the neuro & natural products

👾👾👾

### Anything else you want to explore?

In [ ]:
# example: Random forest regression by property
# unfortuantely, the fingerprint lecture is after this!
# so we will use the properties only

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.inspection import partial_dependence
import pandas as pd

# I added these to my table, magically...
props = ['MW',  'NumHeavyAtoms', 'CLogP', 'HBA', 'HBD',
       'TPSA', 'RotBonds',  'RadiusOfGyration',
       'SphericityIndex', 'NumAromaticRings','FractionCSP3']

X = smol_df[props]
X_scaled = pd.DataFrame(MinMaxScaler().fit_transform(X), columns=props)
y = smol_df.is_neuro.astype(int)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_scaled, y)
feature_importances = pd.Series(model.feature_importances_, index=X_scaled.columns).sort_values(ascending=False)
print(feature_importances)

# ----------------------------------

def plot_partial_dependence(model, X, features):
    fig = go.Figure()
    
    for i, feature in enumerate(features):
        pd_results = partial_dependence(model, X, [feature])
        pd_values = pd_results['average'][0]
        grid_values = pd_results['grid_values'][0]
        
        trace = go.Scatter(
            x=grid_values,
            y=pd_values,
            mode='lines',
            name=feature
        )
        fig.add_trace(trace)
    
    fig.update_layout(title_text='Partial Dependence Plots of properties', 
                      template='plotly_white',
                      yaxis=dict(title='Partial Depencence of total hits'), 
                                 xaxis=dict(title='Value of mol property (MinMaxScaled values)')
                     )
    fig.show()

plot_partial_dependence(model, X_scaled, props)

print('These neuro molecules violate the model!')
sub = smol_df.loc[smol_df.is_neuro]
idx = model.predict(sub[props]) < 0.75
Draw.MolsToGridImage(sub.loc[idx].mol.to_list(),
                     legends=sub.pref_name.to_list(),
                     molsPerRow=5, maxMols=300)

In [ ]:
# That was an example. But I did do a big no-no. What was it?

👾👾👾

# There is a random spike in whatever subset you when with? Why?

👾👾👾 # (see below if confused)

In [ ]:
# You are right, this is because the data is not homogeneous. In the neuronal this is due to benzodiazepam. We could fix it this way:

from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdFingerprintGenerator as rdfpg
from rdkit.Chem import rdMolDescriptors as rdmd

def butina_cluster(mol_list, cutoff=0.35):
    # https://github.com/PatWalters/workshop/blob/master/clustering/taylor_butina.ipynb
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(AllChem.RemoveAllHs(m), 3, nBits=2048) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        dists.extend([1 - x for x in sims])
    mol_clusters = Butina.ClusterData(dists, nfps, cutoff, isDistData=True)
    cluster_id_list = [0] * nfps
    for idx, cluster in enumerate(mol_clusters, 1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list

# howevever, we have not yet added the fingerprints to the dataframe so might want to make a FP column and then do this.

👾👾👾

# this is mad slow for big datasets. Why?

👾👾👾

# To fix it. We could use pytorch / JAX. Or we could do something else?

👾👾👾

In [ ]:
# ## Confusion matrix
# what did we learn about the confusion matrix earlier in the week?

👾👾👾

## Other tables
In ChEMBL there are many tables. Some are more important than others depending on your task.
Due to the size of the database, we will not be able to look at all of them.

In the files of this practical are three exported pickled pandas tables:

```python
sql_query = 'SELECT * FROM ASSAY_TYPE'
pd.read_sql_query(f"{sql_query};", conn).to_pickle('ChEMBL-assay_type.pkl.gz')

activity = pd.read_sql_query(f"SELECT molregno, activity_id, assay_id, units, value, relation FROM ACTIVITIES WHERE type = 'IC50'", conn).to_pickle('ChEMBL-activity.pkl.gz')
molregno2pref_name = df.set_index('molregno').pref_name.to_dict()
activity['pref_name'] = activity.molregno.map(molregno2pref_name)
activity.to_pickle('ChEMBL-activity.pkl.gz')

pd.read_sql_query(f"SELECT assay_id, assay_type, confidence_score FROM ASSAYS;", conn).to_pickle('ChEMBL-assay.pkl.gz')
```
Let's load the to see what they look like.


In [ ]:
import pandas as pd

path_prefix = 'https://github.com/matteoferla/DTC-compchem-practical/raw/refs/heads/main/2024-notebooks/data'
activity_df = pd.read_pkl(f'path_prefix/ChEMBL-activity.pkl.gz')
assay_df = pd.read_pkl(f'path_prefix/ChEMBL-assay.pkl.gz')
assay_type_df = pd.read_pkl(f'path_prefix/ChEMBL-assay_type.pkl.gz')

# remember, chemblid is used for molecules, targets and assays... while molregno is the molecule id

### look at the tables

👾👾👾

In [ ]:
### Scatter plot?
import plotly.express as px

px.scatter(activity_df, x='👾👾👾', y='👾👾👾', color='👾👾👾', hover_data=['👾👾👾'])

In [ ]:
# what about fingerprints?
# https://www.rdkit.org/docs/GettingStartedInPython.html#rdkit-topological-fingerprints
# Now, a fingerprint is a bit vector, which is a series of 0s and 1s.
# we want a matrix that we can pass to `df.DataFrame(matrix, col=cols)` and concatenate (on axis=1) to the existing dataframe (remember to do `.reset_index(drop=True)`).

👾👾👾

# did the model improve?

Other possible questions:

* What does a linear regressor vs RF regressor results look like?
* What is the target with the most data drugs?
* What properties are changing with USAN date?
* 